In [5]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import json
%matplotlib inline

In [8]:
cuisines = []
flavors = []
ingredients = []
name = []
source = []
for filename in os.listdir('data/metadata27638'):
    if filename.endswith('.json'):
        data = json.load(open('data/metadata27638/'+filename))
        cuisines.append(data['attributes']['cuisine'][0])
        flavors.append(data['flavors'])
        ingredientStr = ''
        for ingred in data['ingredientLines']:
            ingredientStr = ingredientStr + ingred
        ingredients.append(ingredientStr)
        name.append(data['name'])
        source.append(data['source']['sourceDisplayName'])
        
df = pd.DataFrame({'cuisine':cuisines,'name':name,'flavors':flavors,'ingredients':ingredients,'source':source})

In [11]:
df.head()

,cuisine,name,flavors,ingredients,source
0,American,Stuffed Cherry Tomatoes,"{'Sweet': 0.8333, 'Sour': 1.0, 'Salty': 0.1667...","2 pints cherry tomatoes1 avocado, peeled and d...",MyRecipes
1,American,Simple Baked Halibut with Bacon Herb Butter,"{'Sweet': 0.0, 'Sour': 0.1667, 'Salty': 0.3333...",1 pound halibut (cut into 2 eight ounce pieces...,PaleOMG
2,American,Honey Whole Wheat Bread,"{'Sweet': 0.0, 'Sour': 0.0, 'Salty': 0.1667, '...",1 cup water (70 degrees to 80 degrees)1/4 cup ...,AllRecipes
3,Italian,Pasta alla Norma,{},3 melanzane / 3 eggplants1 spicchio d’aglio / ...,Eat with Marco
4,American,Plum Upside-Down Cake,"{'Sweet': 1.0, 'Sour': 0.5, 'Salty': 0.5, 'Piq...","2 teaspoons butter6 large red plums, pitted an...",MyRecipes


In [12]:
df['source'].value_counts()

MyRecipes                     7290
AllRecipes                    2521
Martha Stewart                2498
Epicurious                    1803
Serious Eats                   529
Food52                         504
Williams-Sonoma                412
Every Day with Rachael Ray     327
Unilever North America         297
Taste of Home                  294
Chefkoch                       287
Lecker                         240
RecipesPlus                    206
Picky Palate                   159
Food Republic                  142
Skinnytaste                    140
Mizkan Brand                   128
Recipe Girl                    117
Saveur                         115
Simply Recipes                 108
Brown Eyed Baker               105
Leite's Culinaria               98
The Kitchn                      86
Recipe.com                      79
How Sweet It Is                 78
The Pioneer Woman               76
Kalyn's Kitchen                 74
Rasa Malaysia                   68
Jo Cooks            